# 데이터 전처리

## 1) 월별 데이터 정렬 및 결측치 처리
- 2020.03월~ 2020.08월 , 2021.03월 ~ 2021.08월, 2022.06월 데이터 각각 처리  
- 예시: 202105 data

### Hive에서 다운받은 원본데이터 불러오기

In [47]:
data = pd.read_csv('202105_uv.csv')

In [48]:
data = data.iloc[:,1:]

### 지역별, 날짜별, 시간별로 정렬

In [ ]:
data = data.sort_values(['202105_uv.stn', '202105_uv.yyyymmdd', '202105_uv.hhnn'])

### 이상치(-999) 결측치로 변환

In [50]:
data.replace(-999, np.nan, inplace=True)

### 3차 스플라인 보간법을 이용하여 결측치 처리

In [52]:
data = data.fillna(data.interpolate(method='cubic'))

### 정렬 및 결측치 처리된 data csv파일로 저장

In [55]:
data.to_csv('202105.csv',encoding='utf-8-sig')

## 2) 데이터 통합

### 정렬 및 결측치 처리된 월별 데이터 통합 

In [ ]:
import pandas as pd
from glob import glob

file_names = glob("결측치제거data\\*.csv") 
total = pd.DataFrame()

for file_name in file_names:
  temp = pd.read_csv(file_name) 
  temp.rename(columns = lambda x: x.split('_')[-1], inplace = True)
  total = pd.concat([total, temp])

In [ ]:
total.to_csv('2003_2206.csv', encoding='utf-8-sig')

## 3) 데이터에 추가 변수 추가

- 13 : 제주 황해 (한경면)
- 105 : 강원도 강릉시 중앙동 63-3
- 108 : 서울특별시 종로구 송월동 11-2 (사직동)
- 112 : 인천광역시 중구 동인천동 25-59
- 115 : 경상북도 울릉군 울릉읍 사동리 57
- 131 : 충청북도 청주시 흥덕구 복대1동 265-22
- 132 : 충청남도 태안군 안면읍 승언리 산12-1
- 133 : 대전광역시 유성구 온천2동 20-4
- 138 : 경상북도 포항시 남구 송도로
- 143 : 대구광역시 동구 효목1동 산234-26
- 146 : 전라북도 전주시 덕진구 덕진동1가 1416-1
- 152 : 울산광역시 중구 병영2동 산8
- 156 : 광주광역시 북구 운암2동 32-25
- 159 : 부산광역시 중구 대청동 9-305
- 165 : 전라남도 목포시 고하대로 815 (연산동)

### 지역별 추가데이터 병합 후 원본데이터에 병합  
예시: stn-13(한경면), 2020년 data 병합

### 한경면의 2020년 추가 data

In [ ]:
import pandas as pd
from glob import glob

file_names=glob("한경면\\2021\\*.csv") 
extra = pd.DataFrame() 

for file_name in file_names:
  temp = pd.read_csv(file_name)
  temp.rename(columns = lambda x: x.split('_')[-1], inplace = True)
  extra = pd.concat([extra, temp], axis=1)

In [ ]:
extra.columns = [' format: day', 'hour', 'precipitation', ' format: day', 'hour',
       'precipitation_form', ' format: day', 'hour', 'temperature',
       ' format: day', 'hour', 'humidity', ' format: day', 'hour',
       'wind_dir', ' format: day', 'hour', 'velocity']

In [ ]:
extra_2 = extra.drop([' format: day', 'hour'], axis=1, inplace=False)
extra_2 = extra_2.iloc[:4421, :]
extra_3 = extra.iloc[:, :2]
extra_4 = pd.concat([extra_3, extra_2], axis=1)
extra_4 = extra_4.rename(columns = {' format: day':'uv.yyyymmdd', 'hour':'uv.hhnn'})

### 한경면 2020년 data에서 각 월 추출 후 년도월일 변환  
- 예시: 2020 3월 

In [1273]:
extra_2003 = extra_4.iloc[:744, :]
extra_2003

,uv.yyyymmdd,uv.hhnn,precipitation,precipitation_form,temperature,humidity,wind_dir,velocity
0,1,0,0.0,0.0,21.600000,81.0,6.8,155.0
1,1,100,0.0,0.0,22.200001,79.0,6.1,145.0
2,1,200,0.0,0.0,22.799999,80.0,6.1,164.0
3,1,300,0.0,0.0,21.900000,78.0,5.7,182.0
4,1,400,0.0,0.0,22.000000,80.0,4.5,182.0
...,...,...,...,...,...,...,...,...
715,30,1900,0.0,0.0,25.299999,98.0,4.0,130.0
716,30,2000,0.0,0.0,25.500000,98.0,4.2,123.0
717,30,2100,0.0,0.0,26.500000,98.0,4.8,126.0
718,30,2200,0.0,0.0,27.700001,95.0,5.3,130.0


In [1275]:
extra_2003['uv.yyyymmdd'] = extra_2003['uv.yyyymmdd'].astype(int)

for i in range(1, 32):
    extra_2003['uv.yyyymmdd'].replace(i, 20220300+i, inplace=True)
    
extra_2003

,uv.yyyymmdd,uv.hhnn,precipitation,precipitation_form,temperature,humidity,wind_dir,velocity
0,20220301,0,0.0,0.0,21.600000,81.0,6.8,155.0
1,20220301,100,0.0,0.0,22.200001,79.0,6.1,145.0
2,20220301,200,0.0,0.0,22.799999,80.0,6.1,164.0
3,20220301,300,0.0,0.0,21.900000,78.0,5.7,182.0
4,20220301,400,0.0,0.0,22.000000,80.0,4.5,182.0
...,...,...,...,...,...,...,...,...
715,20220330,1900,0.0,0.0,25.299999,98.0,4.0,130.0
716,20220330,2000,0.0,0.0,25.500000,98.0,4.2,123.0
717,20220330,2100,0.0,0.0,26.500000,98.0,4.8,126.0
718,20220330,2200,0.0,0.0,27.700001,95.0,5.3,130.0


### 2020 3월과 동일하게 3~8월 진행 후 2020년 data로 통합

In [1250]:
extra_2020 = pd.concat([extra_2003, extra_2004, extra_2005, 
                        extra_2006, extra_2007, extra_2008])
extra_2020['uv.stn'] = 13
extra_2020

,uv.yyyymmdd,uv.hhnn,precipitation,precipitation_form,temperature,humidity,wind_dir,velocity,uv.stn
0,20200301,0.0,0.0,0.0,12.700000,83.0,6.1,344.0,13
1,20200301,100.0,0.0,0.0,12.400000,88.0,2.2,72.0,13
2,20200301,200.0,0.0,1.0,12.100000,91.0,1.2,263.0,13
3,20200301,300.0,0.0,0.0,12.300000,94.0,4.7,300.0,13
4,20200301,400.0,0.0,0.0,12.200000,94.0,0.0,0.0,13
...,...,...,...,...,...,...,...,...,...
4416,20200831,1900.0,0.0,0.0,25.100000,93.0,2.4,354.0,13
4417,20200831,2000.0,0.0,0.0,25.299999,92.0,2.4,7.0,13
4418,20200831,2100.0,0.0,0.0,25.799999,88.0,1.9,54.0,13
4419,20200831,2200.0,0.0,0.0,26.400000,91.0,3.2,46.0,13


### 추가변수 2020년data csv파일로 저장

In [1251]:
extra_2020.to_csv('extra_2020_13.csv')

### 2020,2021년, 202206 추가변수 동일하게 진행

### 지역별 추가변수 통합

In [2]:
import pandas as pd
from glob import glob

file_names=glob("추가데이터_모음\\*.csv") 
add = pd.DataFrame() 

for file_name in file_names:
  temp = pd.read_csv(file_name) 
  temp.rename(columns = lambda x: x.split('_')[-1], inplace = True)
  add = pd.concat([add, temp]) 
add

,Unnamed: 0,uv.yyyymmdd,uv.hhnn,precipitation,form,temperature,humidity,dir,velocity,uv.stn
0,0,20200301,0.0,0.0,0.0,11.1,57.0,1.5,111.0,105
1,1,20200301,100.0,0.0,0.0,13.0,51.0,2.0,86.0,105
2,2,20200301,200.0,0.0,0.0,12.2,57.0,2.9,106.0,105
3,3,20200301,300.0,0.0,0.0,11.4,65.0,3.3,101.0,105
4,4,20200301,400.0,0.0,0.0,12.0,64.0,4.0,111.0,105
...,...,...,...,...,...,...,...,...,...,...
715,715,20220630,1900.0,0.0,0.0,24.0,92.0,2.1,103.0,165
716,716,20220630,2000.0,0.0,0.0,24.4,91.0,1.6,125.0,165
717,717,20220630,2100.0,0.0,0.0,26.0,84.0,2.3,130.0,165
718,718,20220630,2200.0,0.0,0.0,27.4,77.0,4.8,131.0,165


## 4)기존 data와 병합

In [ ]:
add = add.iloc[:, 1:]

In [34]:
data = pd.read_csv("기상청_결합본\\2003_2206.csv")
data = data.iloc[:, 3:]
data

,uv.yyyymmdd,uv.hhnn,uv.stn,uv.lon,uv.lat,uv.uv,uv.band1,uv.band2,uv.band3,uv.band4,...,uv.band12,uv.band13,uv.band14,uv.band15,uv.band16,uv.solarza,uv.sateza,uv.esr,uv.height,uv.landtype
0,20200301,0,13,126.16000,33.30000,0.0,0.00000,0.00000,-0.00030,0.00015,...,245.40340,264.06961,263.13564,261.93565,253.72902,150.70767,38.96359,-8.52329,71.00,0
1,20200301,0,105,128.89098,37.75147,0.0,0.00000,0.00057,-0.00060,-0.00105,...,248.09458,272.77942,273.03102,272.18208,260.31878,147.70842,43.95463,-8.26618,26.04,3
2,20200301,0,108,126.96500,37.57100,-999.0,0.00000,-0.00057,0.00000,0.00015,...,250.10372,276.15670,276.46287,275.44535,262.34658,147.23785,43.77999,-8.22832,85.50,3
3,20200301,0,112,126.62490,37.47772,0.0,0.00000,-0.00057,0.00030,0.00000,...,249.81751,275.20050,275.51001,274.37866,261.88885,147.21335,43.67046,-8.22680,68.99,3
4,20200301,0,115,130.89864,37.48129,0.0,0.00000,0.00057,0.00000,0.00045,...,248.45659,275.15396,274.70636,273.31842,260.70869,148.51155,43.74521,-8.33183,222.80,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846730,20220625,0,146,127.11900,35.84080,NaN,0.00000,-0.00057,-0.00089,0.00015,...,258.49191,286.87485,287.19605,285.48855,271.35400,119.84431,41.81018,-4.59718,62.90,3
846731,20220625,0,152,129.33472,35.58250,NaN,-0.00057,0.00000,-0.00030,-0.00030,...,263.99872,293.32422,293.51096,291.15861,274.60342,120.42121,41.52495,-4.67124,82.00,2
846732,20220625,0,156,126.89158,35.17294,NaN,0.00000,0.00000,0.00089,-0.00030,...,261.53112,290.14436,290.23278,288.05655,273.15633,120.46896,41.04849,-4.68457,72.38,3
846733,20220625,0,159,129.03203,35.10468,NaN,0.00000,0.00057,0.00030,-0.00075,...,260.95627,287.80066,287.63090,285.01340,270.36623,120.85844,40.96396,-4.73249,69.56,3


In [38]:
data_2 = pd.merge(data, add,how='outer',on=['uv.yyyymmdd', 'uv.hhnn', 'uv.stn'])
data_2

,uv.yyyymmdd,uv.hhnn,uv.stn,uv.lon,uv.lat,uv.uv,uv.band1,uv.band2,uv.band3,uv.band4,...,uv.sateza,uv.esr,uv.height,uv.landtype,precipitation,form,temperature,humidity,dir,velocity
0,20200301,0.0,13,126.16000,33.30000,0.0,0.0,0.00000,-0.0003,0.00015,...,38.96359,-8.52329,71.00,0.0,0.0,0.0,12.7,83.0,6.1,344.0
1,20200301,0.0,105,128.89098,37.75147,0.0,0.0,0.00057,-0.0006,-0.00105,...,43.95463,-8.26618,26.04,3.0,0.0,0.0,11.1,57.0,1.5,111.0
2,20200301,0.0,108,126.96500,37.57100,-999.0,0.0,-0.00057,0.0000,0.00015,...,43.77999,-8.22832,85.50,3.0,0.0,0.0,6.9,65.0,1.6,80.0
3,20200301,0.0,112,126.62490,37.47772,0.0,0.0,-0.00057,0.0003,0.00000,...,43.67046,-8.22680,68.99,3.0,0.0,0.0,5.9,88.0,2.1,140.0
4,20200301,0.0,115,130.89864,37.48129,0.0,0.0,0.00057,0.0000,0.00045,...,43.74521,-8.33183,222.80,4.0,0.0,0.0,8.6,51.0,1.1,238.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848875,20220630,1900.0,165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,24.0,92.0,2.1,103.0
848876,20220630,2000.0,165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,24.4,91.0,1.6,125.0
848877,20220630,2100.0,165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,26.0,84.0,2.3,130.0
848878,20220630,2200.0,165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,27.4,77.0,4.8,131.0


### 제공 데이터는 6월 25일까지 --> 26일~30일 데이터 제거

In [39]:
data_2 = data_2.iloc[:846735, :]
data_2

,uv.yyyymmdd,uv.hhnn,uv.stn,uv.lon,uv.lat,uv.uv,uv.band1,uv.band2,uv.band3,uv.band4,...,uv.sateza,uv.esr,uv.height,uv.landtype,precipitation,form,temperature,humidity,dir,velocity
0,20200301,0.0,13,126.16000,33.30000,0.0,0.00000,0.00000,-0.00030,0.00015,...,38.96359,-8.52329,71.00,0.0,0.0,0.0,12.700000,83.0,6.1,344.0
1,20200301,0.0,105,128.89098,37.75147,0.0,0.00000,0.00057,-0.00060,-0.00105,...,43.95463,-8.26618,26.04,3.0,0.0,0.0,11.100000,57.0,1.5,111.0
2,20200301,0.0,108,126.96500,37.57100,-999.0,0.00000,-0.00057,0.00000,0.00015,...,43.77999,-8.22832,85.50,3.0,0.0,0.0,6.900000,65.0,1.6,80.0
3,20200301,0.0,112,126.62490,37.47772,0.0,0.00000,-0.00057,0.00030,0.00000,...,43.67046,-8.22680,68.99,3.0,0.0,0.0,5.900000,88.0,2.1,140.0
4,20200301,0.0,115,130.89864,37.48129,0.0,0.00000,0.00057,0.00000,0.00045,...,43.74521,-8.33183,222.80,4.0,0.0,0.0,8.600000,51.0,1.1,238.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846730,20220625,0.0,146,127.11900,35.84080,NaN,0.00000,-0.00057,-0.00089,0.00015,...,41.81018,-4.59718,62.90,3.0,0.0,0.0,29.900000,66.0,1.3,163.0
846731,20220625,0.0,152,129.33472,35.58250,NaN,-0.00057,0.00000,-0.00030,-0.00030,...,41.52495,-4.67124,82.00,2.0,0.0,0.0,28.299999,62.0,3.0,232.0
846732,20220625,0.0,156,126.89158,35.17294,NaN,0.00000,0.00000,0.00089,-0.00030,...,41.04849,-4.68457,72.38,3.0,0.0,0.0,27.900000,93.0,2.7,220.0
846733,20220625,0.0,159,129.03203,35.10468,NaN,0.00000,0.00057,0.00030,-0.00075,...,40.96396,-4.73249,69.56,3.0,0.0,0.0,25.600000,84.0,3.6,252.0


### 추가데이터는 1시간 단위, 기존 데이터는 10분 단위 따라서 10분 단위 부분 NaN으로 병합
### 결측치 3차 스플라인 보간법을 이용하여 채우기

In [ ]:
data_2 = data_2.fillna(data_2.interpolate(method='cubic'))

## 5)lon, lat, height column 제거

의미가 동일한 변수를 여러개 사용할 시 의도치 않은 가중치를 부여할 수 있음  
stn(지역)이 동일할 때 lon(경도), lat(위도), height(관측높이)가 동일하므로 제거

In [20]:
total = data_2.drop(['uv.lon','uv.lat','uv.height'], axis=1, inplace=False)

In [21]:
total

,uv.yyyymmdd,uv.hhnn,uv.stn,uv.landtype,uv.uv,uv.band1,uv.band2,uv.band3,uv.band4,uv.band5,...,uv.band16,uv.solarza,uv.sateza,uv.esr,precipitation,form,temperature,humidity,dir,velocity
0,20210601,0.0,13,0.0,0.055064,0.096459,0.094531,0.109731,0.110360,0.633783,...,0.407892,0.788533,0.000000,0.202549,0.012821,0.25,0.804701,0.702941,0.100001,0.187009
1,20210601,10.0,13,0.0,0.055064,0.096459,0.094531,0.109756,0.110338,0.633783,...,0.407974,0.791185,0.000000,0.202549,0.012821,0.25,0.804693,0.702931,0.100002,0.187018
2,20210601,20.0,13,0.0,0.055064,0.096460,0.094493,0.109662,0.110303,0.633854,...,0.410097,0.793147,0.000000,0.202549,0.012821,0.25,0.804685,0.702922,0.100002,0.187027
3,20210601,30.0,13,0.0,0.055064,0.096459,0.094572,0.109829,0.110349,0.633708,...,0.408137,0.794408,0.000000,0.202549,0.012821,0.25,0.804678,0.702912,0.100003,0.187036
4,20210601,40.0,13,0.0,0.055064,0.096459,0.094572,0.109683,0.110393,0.633933,...,0.404062,0.794804,0.000000,0.202549,0.012821,0.25,0.804670,0.702902,0.100003,0.187046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51850,20220624,2320.0,165,3.0,NaN,0.096504,0.094572,0.109683,0.110371,0.633858,...,0.409848,0.750517,0.343474,0.258583,0.012821,0.25,0.885177,0.782178,0.154610,0.488458
51851,20220624,2330.0,165,3.0,NaN,0.096459,0.094572,0.109731,0.110304,0.633783,...,0.410496,0.755707,0.343474,0.258583,0.012821,0.25,0.888249,0.772277,0.148936,0.509695
51852,20220624,2340.0,165,3.0,NaN,0.096504,0.094531,0.109731,0.110349,0.633933,...,0.410388,0.760316,0.343474,0.258583,0.012821,0.25,0.891321,0.762376,0.143262,0.530933
51853,20220624,2350.0,165,3.0,NaN,0.096504,0.094572,0.109756,0.110327,0.633858,...,0.410037,0.764325,0.343474,0.258583,0.012821,0.25,0.894393,0.752475,0.137589,0.552170


## 6)정규화

### 정규화 진행하지 않을 colums

In [ ]:
total1 = total.iloc[:,[0,1,2,3]]

### 정규화 진행할 columns

In [ ]:
total2 = total.iloc[:,4:]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
total2[:] = scaler.fit_transform(total2[:])

In [150]:
data_re = pd.concat([total1,total2],axis=1)
data_re 

,uv.yyyymmdd,uv.hhnn,uv.stn,uv.landtype,uv.uv,uv.band1,uv.band2,uv.band3,uv.band4,uv.band5,...,uv.band11,uv.band12,uv.band13,uv.band14,uv.band15,uv.band16,uv.solarza,uv.sateza,uv.esr,velocity
0,20200301,0,13,0,0.055064,0.096504,0.094572,0.109731,0.110382,0.633708,...,0.415400,0.379140,0.409002,0.396633,0.380553,0.377943,0.975389,0.000000,0.013690,0.288618
15,20200301,10,13,0,0.055064,0.096549,0.094572,0.109829,0.110393,0.633783,...,0.420358,0.383928,0.414027,0.402075,0.386677,0.384443,0.982197,0.000000,0.013690,0.262195
30,20200301,20,13,0,0.055064,0.096549,0.094654,0.109658,0.110360,0.633858,...,0.411070,0.375340,0.405283,0.390247,0.370574,0.367039,0.987896,0.000000,0.013690,0.235772
45,20200301,30,13,0,0.055064,0.096504,0.094531,0.109756,0.110315,0.633933,...,0.397869,0.364041,0.391930,0.376586,0.355647,0.352734,0.992385,0.000000,0.013690,0.209350
60,20200301,40,13,0,0.055064,0.096459,0.094531,0.109683,0.110282,0.633858,...,0.377425,0.344402,0.370796,0.357501,0.336577,0.332328,0.995571,0.000000,0.013690,0.182927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846674,20220624,2320,165,3,NaN,0.096504,0.094572,0.109683,0.110371,0.633858,...,0.442188,0.409779,0.436246,0.425715,0.413246,0.409848,0.750517,0.343474,0.258583,0.147696
846689,20220624,2330,165,3,NaN,0.096459,0.094572,0.109731,0.110304,0.633783,...,0.443951,0.411289,0.437907,0.427126,0.414410,0.410496,0.755707,0.343474,0.258583,0.142276
846704,20220624,2340,165,3,NaN,0.096504,0.094531,0.109731,0.110349,0.633933,...,0.443882,0.411152,0.437865,0.427038,0.414356,0.410388,0.760316,0.343474,0.258583,0.136856
846719,20220624,2350,165,3,NaN,0.096504,0.094572,0.109756,0.110327,0.633858,...,0.442442,0.409814,0.436681,0.426187,0.413641,0.410037,0.764325,0.343474,0.258583,0.131436


## 7) 통합 데이터 나누기
- 학습데이터: 202003~ 202108 (202106~ 202107 제외)  
- 검증데이터: 202106~202107  
- 테스트데이터: 202206

## train, test 나누기

In [ ]:
def abc(n):
    n = n//100
    n= n%10000
    return n

### 학습데이터+ 검증데이터

In [151]:
train=data_re[data_re['uv.yyyymmdd'].apply(abc)!=2206] 

### 테스트데이터

In [152]:
test2=data_re[data_re['uv.yyyymmdd'].apply(abc)==2206]

### 학습데이터

In [ ]:
train_1=train[(train['uv.yyyymmdd'].apply(abc)!=2106) & 
              (train['uv.yyyymmdd'].apply(abc)!=2107)]

### 검증데이터

In [ ]:
train_2=train[(train['uv.yyyymmdd'].apply(abc)==2106) | 
              (train['uv.yyyymmdd'].apply(abc)==2107)

### 각 data csv파일로 저장

In [160]:
train_1.to_csv('train_추가본.csv',encoding='utf-8-sig')

In [161]:
train_2.to_csv('test_추가본.csv',encoding='utf-8-sig')

In [162]:
test2.to_csv('2206_추가본.csv',encoding='utf-8-sig')

# 회귀분석

### 예측할 202206 data를 제외한 나머지 data끼리 회귀분석 진행하여 유의하지 않은 변수들 찾기

In [1]:
import pandas as pd

In [4]:
data = pd.read_csv('train_추가본.csv')
data2 = pd.read_csv('test_추가본.csv')

In [13]:
data3 = pd.concat([data,data2],axis=0)
data3 = data3.iloc[:, 1:]
data3.to_csv('202003-202108.csv', encoding='utf-8-sig')

In [14]:
data = pd.read_csv('202003-202108.csv', )
data

,Unnamed: 0,uv.yyyymmdd,uv.hhnn,uv.stn,uv.landtype,uv.uv,uv.band1,uv.band2,uv.band3,uv.band4,...,uv.band16,uv.solarza,uv.sateza,uv.esr,precipitation,form,temperature,humidity,dir,velocity
0,0,20200301,0.0,13,0.0,0.055064,0.096504,0.094572,0.109731,0.110382,...,0.377943,0.975389,0.000000,0.013690,0.012821,0.25,0.722350,0.831683,0.302128,0.955679
1,1,20200301,10.0,13,0.0,0.055064,0.096549,0.094572,0.109829,0.110393,...,0.384443,0.982197,0.000000,0.013690,0.012821,0.25,0.721774,0.839934,0.274468,0.830102
2,2,20200301,20.0,13,0.0,0.055064,0.096549,0.094654,0.109658,0.110360,...,0.367039,0.987896,0.000000,0.013690,0.012821,0.25,0.721198,0.848185,0.246809,0.704524
3,3,20200301,30.0,13,0.0,0.055064,0.096504,0.094531,0.109756,0.110315,...,0.352734,0.992385,0.000000,0.013690,0.012821,0.25,0.720622,0.856436,0.219149,0.578947
4,4,20200301,40.0,13,0.0,0.055064,0.096459,0.094531,0.109683,0.110282,...,0.332328,0.995571,0.000000,0.013690,0.012821,0.25,0.720046,0.864686,0.191489,0.453370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794875,131755,20210731,2310.0,165,3.0,0.055064,0.096504,0.094572,0.109707,0.110371,...,0.386802,0.776043,0.343474,0.222437,0.012821,0.25,0.889401,0.712871,0.161702,0.235457
794876,131756,20210731,2320.0,165,3.0,0.055064,0.096549,0.094572,0.109731,0.110349,...,0.386741,0.782558,0.343474,0.222437,0.012821,0.25,0.889401,0.712871,0.161702,0.235457
794877,131757,20210731,2330.0,165,3.0,0.055064,0.096504,0.094572,0.109829,0.110293,...,0.382428,0.788488,0.343474,0.222437,0.012821,0.25,0.889401,0.712871,0.161702,0.235457
794878,131758,20210731,2340.0,165,3.0,0.055064,0.096504,0.094572,0.109878,0.110371,...,0.386558,0.793807,0.343474,0.222437,0.012821,0.25,0.889401,0.712871,0.161702,0.235457


In [15]:
import statsmodels.api as sm
x = ['uv.yyyymmdd', 'uv.hhnn', 'uv.stn', 'uv.landtype', 'uv.band1',
       'uv.band2', 'uv.band3', 'uv.band4', 'uv.band5', 'uv.band6', 'uv.band7',
       'uv.band8', 'uv.band9', 'uv.band10', 'uv.band11', 'uv.band12',
       'uv.band13', 'uv.band14', 'uv.band15', 'uv.band16', 'uv.solarza',
       'uv.sateza', 'uv.esr', 'precipitation', 'form', 'temperature',
       'humidity', 'dir', 'velocity']
X = data[x]
X = sm.add_constant(X)
y = data['uv.uv']
res = sm.OLS(y, X).fit()
print(res.summary())


                            OLS Regression Results                            
Dep. Variable:                  uv.uv   R-squared:                       0.727
Model:                            OLS   Adj. R-squared:                  0.727
Method:                 Least Squares   F-statistic:                 7.307e+04
Date:                Sat, 30 Jul 2022   Prob (F-statistic):               0.00
Time:                        17:08:06   Log-Likelihood:             1.2559e+06
No. Observations:              794880   AIC:                        -2.512e+06
Df Residuals:                  794850   BIC:                        -2.511e+06
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.2438      0.266      0.918

유의수준이 0.05일때  p-value가 유의수준보다 큰 변수는 회귀분석에서 유의하지 않은 변수이므로  
yyyymmdd, band15, precipitation 제거

# 모델링

## 1) XGboost
## 2) LGBM
## 3) Randomforest

- 추가데이터 병합한 파일 사용  
- 다중공선성 있는 변수들 제거

#### 홈페이지 rmse검증을 통해 최적의 파라미터로 계속 수정

### 라이브러리 import 및 학습데이터 불러오기

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import GridSearchCV  
from sklearn.model_selection import TimeSeriesSplit

from sklearn.metrics import mean_squared_error
from math import sqrt

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train = pd.read_csv('공모전 모델\\train_추가본.csv', 
                    index_col=0)  
test = pd.read_csv('공모전 모델링\\test_추가본.csv', index_col=0)  


trainX = train.drop(['uv.yyyymmdd', 'uv.band15', 'precipitation', 'uv.uv'], 
                    axis=1, inplace=False)
trainY = train['uv.uv']


testX = test.drop(['uv.yyyymmdd', 'uv.band15', 'precipitation', 'uv.uv'], 
                  axis=1, inplace=False)
testY = test['uv.uv']

### 1. XGBRegressor

In [ ]:
cv = 4

paramGrid = {"subsample" : [0.5, 0.7, 0.9], 'max_depth' : [3, 5, 7, 9,11],
             'n_estimators' : [500, 1000, 10000], 'learning_rate' : [0.01, 0.1, 0.3]}

fit_params={"early_stopping_rounds":100, 
            "eval_metric" : "rmse", 
            "eval_set" : [[testX, testY]]}

model = XGBRegressor(random_state=42, tree_method='gpu_hist', gpu_id=0)

gridsearch = GridSearchCV(model, paramGrid, verbose=1,
                          cv=TimeSeriesSplit(n_splits=cv).get_n_splits([trainX, trainY]))

gridsearch.fit(trainX, trainY, **fit_params)

print(model)
print('GridSearchCV 최고 평균 정확도 수치: {:.4f}'.format(gridsearch.best_score_))
print('GridSearchCV 최적 하이퍼파라미터: ', gridsearch.best_params_)

scores_df = pd.DataFrame(gridsearch.cv_results_)
scores_df

### 최적의 파라미터로 다시 모델링

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

best_model_XGBR = gridsearch.best_estimator_
best_pred = best_model_XGBR.predict(testX)

rmse = sqrt(mean_squared_error(testY, best_pred))

print("훈련 세트 정확도: {:.4f}".format(best_model_XGBR.score(trainX, trainY)))
print("테스트 세트 정확도: {:.4f}".format(best_model_XGBR.score(testX, testY)))
print('{}\n rmse: {:.4f}'.format(best_model_XGBR, rmse))

### 예측할 data 불러오기 및 최적 파라미터 모델로 예측

In [ ]:
real_test = pd.read_csv('공모전 모델링\\2206_추가본.csv', 
                        index_col=0)  
real_testX = real_test.drop(['uv.yyyymmdd', 'uv.band15', 'precipitation', 
                             'uv.uv'], axis=1, inplace=False) 

real_pred = best_model_XGBR.predict(real_testX) 
print(real_pred)

### 2. LGBM

In [ ]:
from lightgbm import LGBMRegressor

cv = 4

paramGrid = {"bagging_fraction" : [0.7], 'num_iterations' : [5000], 'max_depth':[7], 
             'min_data_in_leaf' : [8], 'learning_rate' : [0.03], 'colsample_bytree':[0.72], 
             'max_bin':[255]}

fit_params={"early_stopping_rounds":100,
            "eval_metric" : "rmse", 
            "eval_set" : [[testX, testY]]}


model_LGBMR = LGBMRegressor(random_state=42)


gridsearch_LGBMR = GridSearchCV(model_LGBMR, paramGrid, verbose=1,
                          cv=TimeSeriesSplit(n_splits=cv).get_n_splits([trainX, trainY]))

gridsearch_LGBMR.fit(trainX, trainY, **fit_params)

print(gridsearch_LGBMR)
print('gridsearch_RDFRCV 최고 평균 정확도 수치: {:.4f}'.format(gridsearch_LGBMR.best_score_))
print('gridsearch_RDFRCV 최적 하이퍼파라미터: ', gridsearch_LGBMR.best_params_)

# gridsearch_RDFRCV 객체의 cv_results_ 속성을 데이터 프레임으로 생성
scores_df = pd.DataFrame(gridsearch_LGBMR.cv_results_)
scores_df

### 최적의 파라미터로 다시 모델링

In [ ]:
best_model_LGBMR = gridsearch_LGBMR.best_estimator_
best_pred = best_model_LGBMR.predict(testX)
rmse = sqrt(mean_squared_error(testY, best_pred)) 

print('{}\n rmse: {:.4f}'.format(best_model_LGBMR, rmse))

In [ ]:
real_test = pd.read_csv('공모전 모델링\\2206.csv', index_col=0)  

real_testX = real_test.drop(['uv.uv'], axis=1, inplace=False) 
real_pred = best_model_LGBMR.predict(real_testX) 

### 예측할 data 불러오기 및 최적 파라미터 모델로 예측

In [ ]:
real_test = pd.read_csv('공모전 모델링\\2206_추가본.csv', 
                        index_col=0)  
real_testX = real_test.drop(['uv.yyyymmdd', 'uv.band15', 'precipitation', 
                             'uv.uv'], axis=1, inplace=False) 

real_pred = best_model_LGBMR.predict(real_testX) 
print(real_pred)

### 3. Random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
cv = 4
paramGrid = {"min_samples_leaf" : [8, 12, 18], 'min_samples_split' : [8, 16, 20], 
             'max_depth' : [6, 8, 10, 12], 'n_estimators' : [1000, 10000, 100000]}

model_RDFR = RandomForestRegressor(random_state=42)

gridsearch_RDFR = GridSearchCV(model_RDFR, paramGrid, verbose=1, 
                               cv=TimeSeriesSplit(n_splits=cv).get_n_splits([trainX, trainY]), 
                               scoring='neg_mean_squared_error', 
                               return_train_score=True,  n_jobs=-1)

gridsearch_RDFR.fit(trainX, trainY)

print(model_RDFR)
print('gridsearch_RDFRCV 최고 -mse 수치: {:.4f}'.format(gridsearch_RDFR.best_score_))
print('gridsearch_RDFRCV 최적 하이퍼파라미터: ', gridsearch_RDFR.best_params_)

scores_df = pd.DataFrame(gridsearch_RDFR.cv_results_)
scores_df

### 최적의 파라미터로 다시 모델링

In [ ]:
best_model_RDFR = gridsearch_RDFR.best_estimator_
best_pred = best_model_RDFR.predict(testX)
rmse = sqrt(mean_squared_error(testY, best_pred))

print("훈련 세트 정확도: {:.4f}".format(best_model_RDFR.score(trainX, trainY)))
print("테스트 세트 정확도: {:.4f}".format(best_model_RDFR.score(testX, testY)))

print('{}\n rmse: {:.4f}'.format(best_model_RDFR, rmse))

### 예측할 data 불러오기 및 최적 파라미터 모델로 예측

In [ ]:
real_test = pd.read_csv('공모전 모델링\\2206_추가본.csv', 
                        index_col=0)  
real_testX = real_test.drop(['uv.yyyymmdd', 'uv.band15', 'precipitation', 
                             'uv.uv'], axis=1, inplace=False) 

real_pred = best_model_RDFR.predict(real_testX) 
print(real_pred)

### 여기부터 3가지 모델 모두 동일 과정

### rmse 검증을 위해 정규화 해제

In [ ]:
uv_min = -1.375246e+00
uv_max = 2.360000e+01

real_pred_final = real_pred * (uv_max - uv_min) + uv_min
real_pred_final

uv_df = pd.DataFrame(real_pred_final, columns=['uv.uv'])

### 기존 2206 data에 uv col 제거 후 예측한 uv col 병합

In [ ]:
real_testX2 = real_testX.reset_index()
final = pd.concat([real_testX2, uv_df],axis=1)
final.drop(['index'], axis=1, inplace=True)

### 검증양식으로 예측data 변환

### 검증양식을 위해 제거했던 yyyymmdd column 병합

yymmdd = real_test['uv.yyyymmdd'].reset_index()
yymmdd = yymmdd.iloc[:,1]

final = pd.concat([final, yymmdd],axis=1)

### 양식에 맞게 정렬 후 uv column만 추출 및 csv파일로 저장

In [ ]:
data_2 = final.sort_values(['uv.yyyymmdd','uv.hhnn'])
data_2.tail(30)

uv = data_2['uv.uv']
uv = uv.reset_index()
uv = uv.iloc[:,1:]
test = pd.read_csv('공모전 모델링\\검증양식.csv')
test = test.iloc[:,[0,1]]

final = pd.concat([test,uv],axis=1)
final.columns = ['YearMonthDayHourMinute', 'STN', 'UV']
final.to_csv('공모전 모델링\\pred.csv', encoding='utf-8-sig')

## 최종모델 - LGBM

### 사용파라미터
* bagging_fraction : 0.72 
* num_iterations : 20000 
* max_depth:7 
* min_data_in_leaf : 8 
* learning_rate : 0.03
* colsample_bytree: 0.72
* scale_pos_weight: 1.5 
* lambda_l1: 0.1
* lambda_l2: 0.35
* early_stopping_rounds: 300
* n_splits : 4

In [ ]:
from lightgbm import LGBMRegressor

cv = 4

paramGrid = {"bagging_fraction" : [0.72], 'num_iterations' : [20000], 'max_depth':[7], 'min_data_in_leaf' : [8], 'learning_rate' : [0.03], 'colsample_bytree':[0.72],
              'scale_pos_weight': [1.5], 'lambda_l1':[0.1], 'lambda_l2':[0.35]}

fit_params={"early_stopping_rounds":300,
            "eval_metric" : "rmse", 
            "eval_set" : [[testX, testY]]}


model_LGBMR = LGBMRegressor(random_state=42)


gridsearch_LGBMR = GridSearchCV(model_LGBMR, paramGrid, verbose=1,
                          cv=TimeSeriesSplit(n_splits=cv).get_n_splits([trainX, trainY]))

gridsearch_LGBMR.fit(trainX, trainY, **fit_params)

print(gridsearch_LGBMR)
print('gridsearch_RDFRCV 최고 평균 정확도 수치: {:.4f}'.format(gridsearch_LGBMR.best_score_))
print('gridsearch_RDFRCV 최적 하이퍼파라미터: ', gridsearch_LGBMR.best_params_)

### 그 이후 최적파라미터로 모델링, 검증 데이터로 변환과정 동일